In [18]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from modules import *

In [19]:
data = pd.read_csv("술 리뷰(전처리).csv", encoding='utf-8-sig')
data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
# data.columns = ['date', 'star','doc']
del data['Unnamed: 0']
data

,date,star,text,name
0,2022-10-31,5,지인으로부터 알게된 복순도가 막걸리처음 먹는순간 너무 맛있어서 깜짝놀랐어요원래 술을...,복순도가
1,2022-11-17,5,그냥 차게 마셔도 군내나 잔여물 찌꺼기 느낌이 거의 없이 깔끔담백하고 샴페인처럼 톡...,복순도가
2,2022-10-19,5,지인이 비싼데 고급지고 맛있는 막걸리가있다고 해서하도 궁금하여 한번 시켜봤어요배송도...,복순도가
3,2022-10-28,3,본인은 맛있었는데 아버지는 너무 삭힌거같다는 평이네요 새콤함이 좀 있긴함근데 막걸리...,복순도가
4,2022-10-13,5,복순도가 맛있다고 한참전에 소문만 들었지 먹어본것은 올해가 처음이네요 평소에도 달달...,복순도가
...,...,...,...,...
31979,2022-11-28,5,벌써 두번째 주문입니다 김장때 수육이랑 같이 먹었는데 너무 맛있었고 어른들이 특히 ...,복순도가
31980,2022-11-28,5,술 한잔도 못드시는 어머니도 너무 맛있게 잘 드시네요,복순도가
31981,2022-11-28,5,역시 최고에요최고 최고,복순도가
31982,2022-11-28,5,맛 어요 집에서 싸게 주문해서 먹어서 맛있어요,복순도가


In [20]:
data.text = data.text.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
data.text = data.text.str.replace("\s+", " ") # white space 삭제
data.text

C:\Users\user\AppData\Local\Temp\ipykernel_3352\2834655480.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.text = data.text.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_3352\2834655480.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.text = data.text.str.replace("\s+", " ") # white space 삭제


0        지인으로부터 알게된 복순도가 막걸리처음 먹는순간 너무 맛있어서 깜짝놀랐어요원래 술을...
1        그냥 차게 마셔도 군내나 잔여물 찌꺼기 느낌이 거의 없이 깔끔담백하고 샴페인처럼 톡...
2        지인이 비싼데 고급지고 맛있는 막걸리가있다고 해서하도 궁금하여 한번 시켜봤어요배송도...
3        본인은 맛있었는데 아버지는 너무 삭힌거같다는 평이네요 새콤함이 좀 있긴함근데 막걸리...
4        복순도가 맛있다고 한참전에 소문만 들었지 먹어본것은 올해가 처음이네요 평소에도 달달...
                               ...                        
31979    벌써 두번째 주문입니다 김장때 수육이랑 같이 먹었는데 너무 맛있었고 어른들이 특히 ...
31980                        술 한잔도 못드시는 어머니도 너무 맛있게 잘 드시네요
31981                                         역시 최고에요최고 최고
31982                           맛 어요 집에서 싸게 주문해서 먹어서 맛있어요 
31983                        와 인생막걸리입니다 이제 다른거 못먹어요복순도가 최고
Name: text, Length: 31984, dtype: object

In [21]:
# 검토 후 추가 삭제 문구 적용
data.dropna(inplace=True)

In [22]:
data[data.text.isnull()]

,date,star,text,name


In [23]:
# data = data[data.doc.str.contains('맛|향|달|넘김|부드|단|도수|냄세')]
# data.reset_index(drop=True, inplace=True)

In [24]:
len(data)

31984

In [25]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt

In [26]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
from konlpy.tag import Okt #형태소 분석
from nltk import bigrams
from nltk.util import ngrams
import pandas as pd

from gensim.models.ldamodel import LdaModel
import matplotlib.pyplot as plt
from gensim.models.coherencemodel import CoherenceModel # 토픽 개수 검증
from tqdm import tqdm
# from tqdm.notebook import tqdm
from gensim import corpora
from gensim.models import TfidfModel

import warnings
warnings.filterwarnings(action='ignore')

from modules import *

In [44]:
#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','쓰리다 도어즈','일요일','휴무','수표','층','틀다','차차','드므','맛집','술집','길','가가'
                 '층','개다 성','사진','파다','동영상','아스 론','자다','는','희다 스토리','한','씨', "것","거","게","데",
                 "이다","건","고","되다","되어다","걸","기", "시","네","듯","랍니","중이","얘","스","도도", "나","수","개",
                 "내","기","제","저","인","있다","이렇다", "그렇다","번","위","팅","분","인","링","란","포","두", "진짜",
                 "하다" ,"이다" ,"가다", "이제" ,"들다", "먹다", "보다"]) #불용어 추가

stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]
    # word2=[]
    # word2.append(words)
    # bigr = bigram_(word2)
    # asd = []
    
    # for i in range(len(bigr[2])):
    #     asd.append(bigr[2][i])
    return words

# def bigram_(tokens):
    
#     #document 별 형태소분석기로 분리된 토큰들을 bigram으로 변환(토큰은 string 상태)
#     bigram_stack = [] #변환된 bigram
#     for token in tokens:
#         bigram = bigrams(token)
#         bigram_token = [' '.join(grams) for grams in bigram]
#         bigram_stack.append(bigram_token)
        
#     #bigram  딕셔너리 생성(아이디 매칭)
#     id2word = corpora.Dictionary(bigram_stack) #unigram
#     #id2word.token2id  #아이디가 매칭된 딕셔너리 확인용

#     #딕셔너리에 매칭한 corpus 생성
#     #생성한 bigram 딕셔너리에서 bigram_stack의 토큰을 아이디로 매칭변환
#     texts = bigram_stack 
#     corpus = [id2word.doc2bow(text) for text in texts]
    
#     #gensim으로 tf-idf 처리 #바이그램 요소 하나를 하나의 워드로 처리해서 tf_idf 점수
#     tfidf = TfidfModel(corpus) 
#     corpus_tfidf = tfidf[corpus]
#     tfidf_corpus = [x for x in corpus_tfidf]
#     return tfidf_corpus, corpus ,id2word

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(tokenizer = lambda x: okt_pos_tagging2(x))
bow_vect = vect.fit_transform(data['text'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [46]:
bow_vect.shape

(31984, 9038)

In [47]:
word_count_dict = dict(zip(word_list, count_list))
word_count_dict


{'가가': 61,
 '가감': 1,
 '가걱': 1,
 '가게': 26,
 '가격': 1336,
 '가격떄문': 1,
 '가계': 1,
 '가공': 1,
 '가구': 2,
 '가글': 1,
 '가급적': 1,
 '가기': 8,
 '가까워지다': 2,
 '가까이': 9,
 '가깝다': 31,
 '가끔': 154,
 '가나': 2,
 '가난': 1,
 '가난하다': 1,
 '가내': 1,
 '가늘다': 1,
 '가능': 7,
 '가능성': 2,
 '가능하다': 55,
 '가대': 1,
 '가도': 11,
 '가동': 1,
 '가두': 1,
 '가두다': 1,
 '가득': 48,
 '가득가득': 1,
 '가득하다': 58,
 '가라': 12,
 '가라앉다': 10,
 '가라앉히다': 3,
 '가로': 36,
 '가루': 4,
 '가르다': 94,
 '가리다': 6,
 '가리비': 1,
 '가마': 1,
 '가막': 1,
 '가만': 39,
 '가만히': 2,
 '가면': 11,
 '가물': 12,
 '가미': 7,
 '가바': 1,
 '가방': 9,
 '가버리다': 4,
 '가벼워지다': 1,
 '가볍다': 327,
 '가보': 1,
 '가보다': 32,
 '가복': 6,
 '가봉': 1,
 '가분': 1,
 '가뿐하다': 1,
 '가사': 4,
 '가성': 49,
 '가세': 1,
 '가쇼': 1,
 '가수': 1,
 '가스': 17,
 '가시': 5,
 '가시다': 5,
 '가신': 2,
 '가야': 3,
 '가오니': 1,
 '가온': 1,
 '가요': 37,
 '가요제': 1,
 '가운': 1,
 '가운데': 2,
 '가위': 1,
 '가은': 1,
 '가을': 3,
 '가의': 27,
 '가이거': 2,
 '가이드': 1,
 '가이드북': 1,
 '가인': 1,
 '가작': 1,
 '가장': 83,
 '가절': 1,
 '가정': 3,
 '가제': 4,
 '가져가다': 72,
 '가져다주다': 1,
 '가져오다': 5,
 '가조': 1,
 '가족': 611

In [48]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

In [49]:
print(tf_idf_vect.shape)


(31984, 9038)


In [50]:
print(tf_idf_vect[0])

  (0, 9015)	0.1371820583516196
  (0, 8842)	0.15664253543508722
  (0, 8045)	0.11580229074794311
  (0, 8014)	0.1722016173762457
  (0, 7949)	0.10693736594326066
  (0, 7543)	0.17568945544818282
  (0, 7498)	0.1481230223516065
  (0, 7280)	0.12603791327340036
  (0, 7252)	0.1824198169373692
  (0, 7084)	0.11542005233205058
  (0, 7026)	0.06356219134317531
  (0, 7016)	0.045595355018992756
  (0, 6839)	0.08080444483026562
  (0, 6540)	0.19537531368654093
  (0, 6134)	0.14474428667464473
  (0, 6013)	0.11659062004501561
  (0, 5870)	0.1317860019687791
  (0, 5743)	0.12327270418880462
  (0, 5730)	0.11963965112850289
  (0, 5533)	0.14305288610508207
  (0, 5232)	0.14660095256176162
  (0, 5224)	0.15664253543508722
  (0, 5203)	0.11038013335059767
  (0, 5202)	0.15163787116787522
  (0, 5083)	0.1647706915248541
  :	:
  (0, 4584)	0.17248090285859263
  (0, 4274)	0.1874528337634657
  (0, 4011)	0.10001014430178257
  (0, 3948)	0.110307692421435
  (0, 3636)	0.07301777291539253
  (0, 2979)	0.07789725262820386
  (0, 2733

In [51]:
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100]+'...')

{7252: '지인', 5203: '알', 3636: '복', 4622: '순도', 2573: '막걸리', 7543: '처음', 4620: '순간', 1366: '너무', 2733...


In [52]:
def rating_to_label(rating):
    if rating > 3:
        return 1
    else:
        return 0

data['y'] = data['star'].apply(lambda x : rating_to_label(x))
len(data[data['y']==0])

1749

In [53]:
from sklearn.model_selection import train_test_split

x = tf_idf_vect
y = data['y']

# positive_random_idx = data[data['y']==1].sample(48, random_state=12).index.tolist()
# negative_random_idx = data[data['y']==0].sample(48, random_state=12).index.tolist()

# random_idx = positive_random_idx + negative_random_idx
# x = tf_idf_vect[random_idx]
# y = data['y'][random_idx]


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)


In [54]:
x.shape,y.shape

((31984, 9038), (31984,))

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# fit in training set
lr = LogisticRegression()
lr.fit(x_train, y_train)

# predict in test set
y_pred = lr.predict(x_test)

In [56]:
print('accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('precision: %.4f' % precision_score(y_test, y_pred))
print('recall: %.4f' % recall_score(y_test, y_pred))
print('F1: %.4f' % f1_score(y_test, y_pred))


accuracy: 0.9452
precision: 0.9478
recall: 0.9968
F1: 0.9717


In [57]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)
coef_neg_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = False)
coef_pos_index



[(5.489109318959452, 2733),
 (2.9170734957652753, 7016),
 (2.5769740413342346, 3711),
 (2.507626282986461, 7662),
 (2.4967725915557057, 6483),
 (2.348885420215767, 1092),
 (2.036111581589789, 5583),
 (1.9046541566386344, 2629),
 (1.8368858368295764, 6718),
 (1.75645574656784, 5769),
 (1.4275774786787527, 4274),
 (1.4249704628945614, 7080),
 (1.4207452530133864, 5833),
 (1.3959851841571596, 3636),
 (1.386228504140855, 8629),
 (1.3669315551537877, 6542),
 (1.3515729653474673, 5125),
 (1.3194386440218204, 781),
 (1.2384596394560887, 1736),
 (1.2294355450448688, 4622),
 (1.218180956939797, 1024),
 (1.2130861210075061, 3701),
 (1.1715129407800753, 7677),
 (1.145227004574611, 435),
 (1.1345536549956927, 2197),
 (1.1220993894502975, 4483),
 (1.1058525778643753, 2099),
 (1.0983816804404367, 4333),
 (1.0715601875768475, 7850),
 (1.0658405117242247, 7798),
 (1.0525373368847148, 6364),
 (1.0110098492264081, 6901),
 (1.0050091548324502, 1186),
 (1.0039366715999234, 4654),
 (0.9983828618614449, 866

In [58]:
for coef in coef_pos_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])


맛있다 5.489109318959452
좋다 2.9170734957652753
부드럽다 2.5769740413342346
최고 2.507626282986461
잇다 2.4967725915557057
꼼꼼하다 2.348885420215767
역시 2.036111581589789
만족하다 1.9046541566386344
적당하다 1.8368858368295764
와인 1.75645574656784
샴페인 1.4275774786787527
주문 1.4249704628945614
요 1.4207452530133864
복 1.3959851841571596
항상 1.386228504140855
자주 1.3669315551537877
아주 1.3515729653474673
귀엽다 1.3194386440218204
달콤하다 1.2384596394560887
순도 1.2294355450448688


In [59]:
for coef in coef_neg_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])


별로 -3.594530408715869
셔 -3.429812830427952
아니다 -3.3022664954766556
시큼하다 -3.112246501179455
실망 -3.009634296863986
신맛 -2.9612922261886974
식초 -2.8078474132205375
그냥 -2.6990767476643933
달다 -2.5919674737544645
밍밍 -2.2650743063847303
다시다 -2.248925380856438
맛없다 -2.2453568201705183
루 -2.207646303960886
기대하다 -2.1296744257823197
입맛 -2.104601387624342
모르다 -2.0562636592255075
비추다 -1.9536026771909736
맞다 -1.9248403626616177
스럽다 -1.8740565174942243
대비 -1.8213357846051883


In [60]:
for coef in coef_neg_index[:]:
    print(invert_index_vectorizer[coef[1]], coef[0])

별로 -3.594530408715869
셔 -3.429812830427952
아니다 -3.3022664954766556
시큼하다 -3.112246501179455
실망 -3.009634296863986
신맛 -2.9612922261886974
식초 -2.8078474132205375
그냥 -2.6990767476643933
달다 -2.5919674737544645
밍밍 -2.2650743063847303
다시다 -2.248925380856438
맛없다 -2.2453568201705183
루 -2.207646303960886
기대하다 -2.1296744257823197
입맛 -2.104601387624342
모르다 -2.0562636592255075
비추다 -1.9536026771909736
맞다 -1.9248403626616177
스럽다 -1.8740565174942243
대비 -1.8213357846051883
버리다 -1.8207648696423708
보통 -1.781348365256863
강하다 -1.715265036612379
심하다 -1.7092386538893747
상한 -1.687744945135064
넘기다 -1.6750763764553374
시어 -1.6533474802873376
건지다 -1.647395557692022
비싸다 -1.6292198501002049
동네 -1.625900673547685
차이 -1.614436481552355
음 -1.5857489165449363
가격 -1.555113909968432
당황 -1.545899606855181
아주아주 -1.530851070552024
적어지다 -1.5026254051594847
개인 -1.4748821377484551
개취 -1.466726133017518
별루 -1.4391108676269484
밋밋하다 -1.434234292703337
확인 -1.3939716102882649
정도 -1.3870004251273584
많이 -1.3847657447067672
스타일 -1.367